<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-19 15:37:02
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: -2.33 L (-1.63%)
Current PnL: -33.51 L (-20.95%)
CY Booked + Current PnL: -17.83 L (-11.15%)
-------------------
Total profit:  97.90 K
Total loss:  -34.49 L
-------------------
Total Booked + Current PnL: 9.15 L (6.88%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.13%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 1.00 C (71.05%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 2.99%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.85,1.27,20.04,21.57,19885.0,1245.0,99225.0,397.04,-12.83,54.0,H-SC,5.05,167.0,0.06,0.72,44.80,XR,ATH,FINANCE
18,COALINDIA,-0.44,2.34,14.98,17.67,23066.0,3516.0,153976.0,478.51,23.39,49.0,L-LC,11.46,185.0,0.15,1.11,23.60,XY25,ATH,MINING
77,TTKPRESTIG,-0.57,-28.21,39.42,0.09,28521.0,-28426.0,72351.0,770.00,67.98,37.0,M-SC,9.45,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
36,ICICIGI,-1.21,2.55,16.37,19.34,33594.0,5107.0,205216.0,2252.93,-13.70,59.0,X-MC,2.61,66.0,0.15,1.48,18.90,X40,ATH,INSURANCE
51,MEDANTA,-2.85,-4.84,30.39,24.08,36714.0,-6142.0,120808.0,1486.00,-5.52,51.0,X-SC,10.65,87.0,-0.17,0.87,10.65,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LAOPALA,2.71,-39.45,129.69,39.09,102961.0,-51715.0,79390.0,464.00,73.50,55.0,H-SC,14.71,154.0,-0.50,0.57,6.16,AR,NTT,CERAMICS
14,BSOFT,2.70,-30.03,115.49,50.79,113662.0,-42236.0,98417.0,831.70,-10.32,40.0,H-SC,6.78,172.0,-0.37,0.71,14.59,XR,ATH,IT
43,IRCTC,0.91,-24.18,101.72,52.93,208614.0,-65421.0,205087.0,1269.00,-43.95,50.0,H-MC,12.42,119.0,-0.31,1.48,3.56,XY24,BTT,TRAVEL
30,HAPPSTMNDS,0.08,-51.71,282.76,84.85,175017.0,-66269.0,61896.0,1480.71,-42.53,39.0,H-SC,25.71,152.0,-0.38,0.45,4.02,AR,ATH,IT
69,SURYODAY,-0.02,-29.26,73.93,23.03,93650.0,-52397.0,126674.0,216.00,38.16,32.0,H-SC,3.37,166.0,-0.56,0.91,25.69,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,-4.96,-14.63,70.52,45.57,118673.0,-28849.0,168283.0,844.0,3645.95,59.0,M-SC,4.13,248.0,-0.24,1.22,27.35,XY24,NTT,AUTO
25,EASEMYTRIP,-4.26,-49.12,186.33,45.70,167639.0,-86846.0,89969.0,26.4,-6.64,74.0,M-SC,21.39,204.0,-0.52,0.65,48.23,XY24,NTT,TRAVEL
19,COFFEEDAY,-4.20,-46.43,152.53,35.27,92778.0,-52723.0,60826.0,80.0,-59.76,46.0,L-SC,14.48,266.0,-0.57,0.44,48.87,XR,NTT,HOTELS
79,VAIBHAVGBL,-3.73,-26.60,119.03,60.77,159689.0,-48616.0,134159.0,521.0,54.82,50.0,H-SC,2.18,160.0,-0.30,0.97,21.63,XR,NTT,JEWELLERY
5,ANGELONE,-3.17,2.77,20.86,24.20,65438.0,8450.0,313700.0,3033.0,49.34,44.0,X-SC,3.34,97.0,0.13,2.27,31.38,X40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-2.22,1.54,104.76,107.91,157054.0,2274.0,149918.0,1641.55,-8.68,58.0,M-SC,8.54,220.0,0.01,1.08,12.79,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.57,-28.21,39.42,0.09,28521.0,-28426.0,72351.0,770.00,67.98,37.0,M-SC,9.45,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,-0.93,-30.25,89.30,32.03,82111.0,-39885.0,91950.0,9161.00,-12.80,34.0,H-SC,7.61,158.0,-0.49,0.66,2.99,OX40N,NTT,IT
66,SIS,-2.99,-30.70,74.98,21.26,57694.0,-34086.0,76946.0,528.00,1806.74,32.0,H-SC,4.35,168.0,-0.59,0.56,6.08,OX40N,NTT,MISC
40,INDIGOPNTS,-0.92,-31.41,45.83,0.02,54869.0,-54837.0,119722.0,1408.00,82.48,29.0,M-SC,9.26,222.0,-1.00,0.86,4.14,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.44,2.34,14.98,17.67,23066.0,3516.0,153976.0,478.51,23.39,49.0,L-LC,11.46,185.0,0.15,1.11,23.60,XY25,ATH,MINING
26,FINCABLES,-2.22,1.54,104.76,107.91,157054.0,2274.0,149918.0,1641.55,-8.68,58.0,M-SC,8.54,220.0,0.01,1.08,12.79,OX40N,ATH,CABLES
50,MASFIN,-1.85,1.27,20.04,21.57,19885.0,1245.0,99225.0,397.04,-12.83,54.0,H-SC,5.05,167.0,0.06,0.72,44.80,XR,ATH,FINANCE
1,ABB,-2.75,4.94,38.72,45.57,106300.0,12917.0,274536.0,7934.00,-32.57,59.0,H-MC,5.04,125.0,0.12,1.98,21.91,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-2.75,4.94,38.72,45.57,106300.0,12917.0,274536.0,7934.00,-32.57,59.0,H-MC,5.04,125.0,0.12,1.98,21.91,AR,NTT,ELECTRICAL
50,MASFIN,-1.85,1.27,20.04,21.57,19885.0,1245.0,99225.0,397.04,-12.83,54.0,H-SC,5.05,167.0,0.06,0.72,44.80,XR,ATH,FINANCE
26,FINCABLES,-2.22,1.54,104.76,107.91,157054.0,2274.0,149918.0,1641.55,-8.68,58.0,M-SC,8.54,220.0,0.01,1.08,12.79,OX40N,ATH,CABLES
39,INDIAMART,-0.87,-7.16,122.66,106.72,140453.0,-8830.0,114506.0,4810.62,-58.20,42.0,H-SC,7.89,140.0,-0.06,0.83,14.55,AR,ATH,MISC
85,ZYDUSLIFE,-0.84,-4.84,42.53,35.63,84818.0,-10144.0,199430.0,1286.17,-17.12,50.0,H-MC,5.28,120.0,-0.12,1.44,11.02,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-0.23,-6.70,43.89,34.24,128122.0,-20972.0,291916.0,1972.00,-26.71,24.0,X-LC,3.93,3.0,-0.16,2.11,1.02,X40,NTT,IT
33,HCLTECH,-1.14,-5.87,31.56,23.83,71866.0,-14211.0,227713.0,1908.19,-1.16,26.0,X-LC,4.36,8.0,-0.20,1.64,9.25,X40,ATH,IT
73,TCS,-0.63,-23.25,61.01,23.58,202590.0,-100569.0,332060.0,4311.59,-33.46,26.0,X-LC,4.83,2.0,-0.50,2.40,0.00,X40,ATH,IT
8,AWL,-1.43,-35.25,142.77,57.19,301483.0,-114960.0,211167.0,485.00,-63.00,27.0,X-SC,17.41,81.0,-0.38,1.52,0.00,XY24,NTT,FMCG
13,BERGEPAINT,-0.64,-18.51,48.65,21.14,90132.0,-42072.0,185267.0,680.00,-20.29,28.0,X-MC,12.80,68.0,-0.47,1.34,0.00,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-1.93,2.63,17.67,20.77,42461.0,6157.0,240301.0,1644.00,-8.94,54.0,X-MC,1.16,64.0,0.15,1.74,10.23,X40N,NTT,BREWERIES
21,DABUR,-1.75,-1.92,46.52,43.70,113183.0,-4768.0,243300.0,735.00,-10.97,42.0,X-MC,1.75,70.0,-0.04,1.76,12.84,XY24,BTT,FMCG
65,SIEMENS,-2.53,-16.52,50.35,25.51,78221.0,-30740.0,155355.0,4671.50,30.44,49.0,H-LC,2.01,51.0,-0.39,1.12,14.48,AR,ATH,ELECTRICAL
36,ICICIGI,-1.21,2.55,16.37,19.34,33594.0,5107.0,205216.0,2252.93,-13.70,59.0,X-MC,2.61,66.0,0.15,1.48,18.90,X40,ATH,INSURANCE
75,TMPV,-1.87,-16.51,59.70,33.33,162388.0,-53793.0,272007.0,600.00,-82.53,56.0,X-LC,2.80,4.0,-0.33,1.96,11.20,XY24,NTT,AUTO


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,-1.40,-32.02,141.47,64.15,228842.0,-76191.0,161760.0,3906.00,-48.50,36.0,X-SC,8.69,85.0,-0.33,1.17,0.0,XY24,BTT,CEMENT
8,AWL,-1.43,-35.25,142.77,57.19,301483.0,-114960.0,211167.0,485.00,-63.00,27.0,X-SC,17.41,81.0,-0.38,1.52,0.0,XY24,NTT,FMCG
12,BATAINDIA,-1.37,-46.75,153.91,35.20,105453.0,-60154.0,68516.0,2096.00,-10.18,35.0,X-SC,21.63,91.0,-0.57,0.49,0.0,X40,NTT,FOOTWEAR
13,BERGEPAINT,-0.64,-18.51,48.65,21.14,90132.0,-42072.0,185267.0,680.00,-20.29,28.0,X-MC,12.80,68.0,-0.47,1.34,0.0,XY24,NTT,PAINTS
53,PGHH,-0.66,-13.69,52.95,32.01,97580.0,-29232.0,184288.0,17616.87,-41.63,33.0,X-MC,9.31,57.0,-0.30,1.33,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.12,-30.71,106.16,42.84,47817.0,-19964.0,45042.0,424.00,-54.82,49.0,X-SC,18.25,80.0,-0.42,0.33,8.34,XY24,NTT,MISC
12,BATAINDIA,-1.37,-46.75,153.91,35.20,105453.0,-60154.0,68516.0,2096.00,-10.18,35.0,X-SC,21.63,91.0,-0.57,0.49,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-1.96,-52.68,221.88,52.31,152400.0,-76474.0,68686.0,1176.00,-49.67,43.0,X-SC,13.97,92.0,-0.50,0.50,2.97,X40N,NTT,FOOTWEAR
51,MEDANTA,-2.85,-4.84,30.39,24.08,36714.0,-6142.0,120808.0,1486.00,-5.52,51.0,X-SC,10.65,87.0,-0.17,0.87,10.65,XY24,NTT,HEALTHCARE
35,HONAUT,-0.88,-23.62,87.64,43.32,109024.0,-38472.0,124400.0,58357.33,-32.99,37.0,X-SC,11.44,90.0,-0.35,0.90,1.01,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.94,-14.13,38.65,19.06,112895.0,-48070.0,292096.0,452.00,-52.69,50.0,X-LC,18.34,1.0,-0.43,2.11,5.76,X40,NTT,FMCG
73,TCS,-0.63,-23.25,61.01,23.58,202590.0,-100569.0,332060.0,4311.59,-33.46,26.0,X-LC,4.83,2.0,-0.50,2.40,0.00,X40,ATH,IT
42,INFY,-0.23,-6.70,43.89,34.24,128122.0,-20972.0,291916.0,1972.00,-26.71,24.0,X-LC,3.93,3.0,-0.16,2.11,1.02,X40,NTT,IT
75,TMPV,-1.87,-16.51,59.70,33.33,162388.0,-53793.0,272007.0,600.00,-82.53,56.0,X-LC,2.80,4.0,-0.33,1.96,11.20,XY24,NTT,AUTO
33,HCLTECH,-1.14,-5.87,31.56,23.83,71866.0,-14211.0,227713.0,1908.19,-1.16,26.0,X-LC,4.36,8.0,-0.20,1.64,9.25,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-1.83,-31.54,94.10,32.89,49740.0,-24348.0,52859.0,1800.00,-348.60,55.0,L-MC,11.12,258.0,-0.49,0.38,45.57,XR,NTT,BANKS
67,SONACOMS,-0.70,-8.65,52.26,39.09,48293.0,-8750.0,92409.0,804.02,-27.63,61.0,M-MC,7.60,193.0,-0.18,0.67,30.73,AR,ATH,AUTO
50,MASFIN,-1.85,1.27,20.04,21.57,19885.0,1245.0,99225.0,397.04,-12.83,54.0,H-SC,5.05,167.0,0.06,0.72,44.80,XR,ATH,FINANCE
18,COALINDIA,-0.44,2.34,14.98,17.67,23066.0,3516.0,153976.0,478.51,23.39,49.0,L-LC,11.46,185.0,0.15,1.11,23.60,XY25,ATH,MINING
7,ATULAUTO,-4.96,-14.63,70.52,45.57,118673.0,-28849.0,168283.0,844.00,3645.95,59.0,M-SC,4.13,248.0,-0.24,1.22,27.35,XY24,NTT,AUTO


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-4.26,-49.12,186.33,45.70,167639.0,-86846.0,89969.0,26.40,-6.64,74.0,M-SC,21.39,204.0,-0.52,0.65,48.23,XY24,NTT,TRAVEL
11,BANDHANBNK,-1.32,-18.48,136.03,92.42,308911.0,-51469.0,227090.0,400.00,94.68,67.0,H-SC,6.68,174.0,-0.17,1.64,28.12,XY24,NTT,BANKS
67,SONACOMS,-0.70,-8.65,52.26,39.09,48293.0,-8750.0,92409.0,804.02,-27.63,61.0,M-MC,7.60,193.0,-0.18,0.67,30.73,AR,ATH,AUTO
1,ABB,-2.75,4.94,38.72,45.57,106300.0,12917.0,274536.0,7934.00,-32.57,59.0,H-MC,5.04,125.0,0.12,1.98,21.91,AR,NTT,ELECTRICAL
7,ATULAUTO,-4.96,-14.63,70.52,45.57,118673.0,-28849.0,168283.0,844.00,3645.95,59.0,M-SC,4.13,248.0,-0.24,1.22,27.35,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.68
1,25,45.34
2,50,77.22


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.35
MC    29.90
LC    24.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.05
X40      23.99
X40N     14.89
AR        8.86
XY25      8.67
XR        8.55
OX40N     7.12
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.55
H-SC    22.22
X-LC    21.44
X-SC    11.61
M-SC    10.77
H-MC     4.90
M-MC     2.07
H-LC     1.12
L-LC     1.11
M-LC     1.06
L-SC     0.75
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.52,-14.75,53.57
FINANCE,13.10,-15.11,60.41
IT,11.72,-35.03,102.70
MISC,6.93,-35.51,90.95
ELECTRICAL,6.12,-10.75,51.16
PAINTS,5.03,-32.53,52.22
INSURANCE,4.81,-1.75,36.57
PHARMA,4.07,-4.20,37.00
AUTO,3.85,-17.16,61.83


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3380479.0,21
X40,1397588.0,15
AR,1344670.0,10
XR,1306869.0,12
X40N,1062316.0,10
OX40N,804102.0,10
XY25,415634.0,6
SR,301131.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3713538.0,24
M-SC,1424619.0,14
X-MC,1392219.0,15
X-LC,1303869.0,12
X-SC,1196514.0,10
H-MC,399732.0,3
L-SC,187026.0,2
M-LC,145103.0,1
M-MC,99142.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1263622.0      6
           AR         952622.0      5
           XR         839892.0      7
M-SC       XY24       817264.0      6
X-LC       X40        664186.0      6
X-SC       XY24       614856.0      4
X-MC       X40        554085.0      7
           X40N       427361.0      4
X-SC       X40N       402341.0      4
H-SC       OX40N      356271.0      4
M-SC       OX40N      353583.0      5
H-SC       SR         301131.0      2
X-LC       XY24       272808.0      2
           X40N       232614.0      2
H-MC       XY24       208614.0      1
X-MC       XY25       207458.0      2
           XY24       203315.0      2
H-MC       AR         191118.0      2
M-SC       XR         179356.0      2
X-SC       X40        179317.0      2
M-LC       XR         145103.0      1
X-LC       XY25       134261.0      2
L-SC       OX40N       94248.0      1
           XR          92778.0      1
H-LC       AR          78221.0      1
M-SC       AR          74416.0      1
M-MC       XY25        50849.0      1
L-MC       XR          49740.0      1
M-MC       AR          48293.0      1
L-LC       XY25        23066.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
